# Rashomon + 33/66/100% Features + SHAP & LIME

**Plantilla reproducible** para estudiar el efecto Rashomon con tres presupuestos de atributos (33%, 66%, 100%),
calculando el *ε-Rashomon set* (WithinEpsilon), métricas de **volumen** (conteo), **complejidad**, y **estabilidad explicativa** con **SHAP** y **LIME**.

**Qué hace:**
1. Rankea atributos y define subconjuntos 33/66/100.
2. Entrena múltiples modelos y selecciona los que están *within-ε* del mejor (por AUC o pérdida).
3. Calcula métricas Rashomon (conteo, complejidad) y **estabilidad de explicaciones** (Kendall τ, Jaccard@k, distancia entre vectores de importancias SHAP; varianza LIME).
4. Grafica **Rashomon curves** (rendimiento vs complejidad) y **Stability vs Complejidad**.

**Modelos incluidos (cuando las dependencias están disponibles):** QDA, XGB, GBM (sklearn), ERT, AdaBoost, MLP, Decision Tree, Logistic Regression, Random Forest, SVM, KNN, Bagging.

> **Nota**: Este notebook intenta importar `shap` y `lime`. Si no los tienes, instálalos:
```bash
pip install shap lime
```
Además, para XGBoost:
```bash
pip install xgboost
```


In [ ]:
# ===============================
# 0) Imports y utilidades
# ===============================
import numpy as np
import pandas as pd
from itertools import product
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.feature_selection import mutual_info_classif

# Modelos clásicos
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier
)

# XGBoost opcional
try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

# SHAP & LIME opcionales
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False
try:
    from lime.lime_tabular import LimeTabularExplainer
    LIME_AVAILABLE = True
except Exception:
    LIME_AVAILABLE = False

import matplotlib.pyplot as plt

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print({'SHAP_AVAILABLE': SHAP_AVAILABLE, 'LIME_AVAILABLE': LIME_AVAILABLE, 'XGB_AVAILABLE': XGB_AVAILABLE})

## 1) Carga de datos
Reemplaza la sección **B)** con tu dataset propio.


In [ ]:
# ============================================
# 1A) Dataset sintético de ejemplo (binario)
# ============================================
X, y = make_classification(
    n_samples=4000, n_features=20, n_informative=8, n_redundant=4, n_repeated=0,
    n_classes=2, weights=[0.55,0.45], class_sep=1.2, random_state=RANDOM_STATE
)
feature_names = [f'f{i}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
print(df.shape, df['target'].value_counts(normalize=True).round(3))

In [ ]:
# ==========================================================
# 1B) (Opcional) Reemplaza por tu CSV
# df = pd.read_csv('ruta/a/tu_dataset.csv')
# y_col = 'target'  # Ajusta el nombre de la columna objetivo
# feature_names = [c for c in df.columns if c != y_col]
# X = df[feature_names].values
# y = df[y_col].values
# ==========================================================

## 2) Train/Valid split y escalado opcional


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=RANDOM_STATE
)

# Escalado para modelos sensibles a la escala
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_valid_s = scaler.transform(X_valid)

## 3) Ranking de atributos y subconjuntos 33/66/100
Combinamos **mutual information** con importancias de un **árbol extra-aleatorio** para robustez del ranking.

In [ ]:
def feature_ranking(X_tr, y_tr, feature_names):
    # Mutual Information (escala 0..1)
    mi = mutual_info_classif(X_tr, y_tr, random_state=RANDOM_STATE)
    # Importancia de ExtraTrees
    et = ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
    et.fit(X_tr, y_tr)
    et_imp = et.feature_importances_
    # Agregado
    score = 0.5*mi + 0.5*et_imp
    rk = pd.DataFrame({'feature': feature_names, 'score': score}).sort_values('score', ascending=False)
    rk['Rank'] = np.arange(1, len(feature_names)+1)
    return rk

ranking_df = feature_ranking(X_train, y_train, feature_names)
display(ranking_df.head(10))

m = len(feature_names)
k33 = max(1, int(np.ceil(0.33*m)))
k66 = max(1, int(np.ceil(0.66*m)))
S33 = ranking_df['feature'].iloc[:k33].tolist()
S66 = ranking_df['feature'].iloc[:k66].tolist()
S100 = ranking_df['feature'].tolist()
S_map = {'33%': S33, '66%': S66, '100%': S100}
print({'k33': k33, 'k66': k66, 'm': m})

## 4) Definición de modelos y grillas simples
Cada modelo tiene una grilla corta de hiperparámetros. El *Rashomon set* se construirá con **reentrenos × grilla**.

In [ ]:
def get_models_dict():
    models = {
        'LogReg': (LogisticRegression(max_iter=500), {'C':[0.1,1,3], 'penalty':['l2'], 'solver':['lbfgs']}),
        'QDA': (QDA(), {}),
        'KNN': (KNeighborsClassifier(), {'n_neighbors':[3,5,11]}),
        'SVM': (SVC(probability=True), {'C':[0.5,1,3], 'kernel':['rbf']}),
        'DT': (DecisionTreeClassifier(random_state=RANDOM_STATE), {'max_depth':[2,3,4,5]}),
        'RF': (RandomForestClassifier(random_state=RANDOM_STATE), {'n_estimators':[150], 'max_depth':[None,5,8]}),
        'ERT': (ExtraTreesClassifier(random_state=RANDOM_STATE), {'n_estimators':[200], 'max_depth':[None,5,8]}),
        'GBM': (GradientBoostingClassifier(random_state=RANDOM_STATE), {'n_estimators':[120], 'max_depth':[2,3]}),
        'ADA': (AdaBoostClassifier(random_state=RANDOM_STATE), {'n_estimators':[150], 'learning_rate':[0.5,1.0]}),
        'Bagging': (BaggingClassifier(random_state=RANDOM_STATE), {'n_estimators':[60]}),
        'MLP': (MLPClassifier(max_iter=400, random_state=RANDOM_STATE), {'hidden_layer_sizes':[(50,), (50,50)], 'alpha':[0.0001, 0.001]}),
    }
    if XGB_AVAILABLE:
        models['XGB'] = (XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE),
                         {'n_estimators':[200], 'max_depth':[3,5], 'learning_rate':[0.1,0.2]})
    return models

models_dict = get_models_dict()
list(models_dict.keys())

## 5) Bucle Rashomon: entrenamiento, selección WithinEpsilon y complejidad
Se define **métrica objetivo** (por defecto AUC). Se calcula el mejor modelo por subconjunto y luego el **ε‑level set**.

In [ ]:
def model_complexity(model):
    # Heurística simple de complejidad por tipo
    from sklearn.base import ClassifierMixin
    label = model.__class__.__name__
    try:
        if label in ['DecisionTreeClassifier']:
            return getattr(model, 'tree_', None).node_count if hasattr(model, 'tree_') else np.nan
        if label in ['RandomForestClassifier','ExtraTreesClassifier','GradientBoostingClassifier','AdaBoostClassifier','BaggingClassifier']:
            return len(getattr(model, 'estimators_', []))
        if label in ['LogisticRegression']:
            return np.count_nonzero(getattr(model, 'coef_', np.array([[]])))
        if label in ['MLPClassifier']:
            return sum(w.size for w in model.coefs_)
        if label in ['SVC']:
            return getattr(model, 'support_vectors_', np.array([])).shape[0]
        if label in ['KNeighborsClassifier']:
            return getattr(model, 'n_neighbors', np.nan)
        if label in ['QuadraticDiscriminantAnalysis']:
            return np.nan
        if label in ['XGBClassifier']:
            return getattr(model, 'n_estimators', np.nan)
    except Exception:
        return np.nan
    return np.nan

def fit_and_score(clf, Xtr, ytr, Xva, yva, use_scaled=False):
    # algunos modelos usan X escalado
    if use_scaled:
        clf.fit(Xtr, ytr)
        pred_proba = clf.predict_proba(Xva)[:,1] if hasattr(clf, 'predict_proba') else clf.decision_function(Xva)
    else:
        clf.fit(Xtr, ytr)
        pred_proba = clf.predict_proba(Xva)[:,1] if hasattr(clf, 'predict_proba') else clf.decision_function(Xva)
    auc = roc_auc_score(yva, pred_proba)
    return auc

def build_rashomon_set(S_name, features, epsilon=0.01, metric='AUC', seeds=[0,1,2]):
    cols_idx = [feature_names.index(f) for f in features]
    Xtr = X_train[:, cols_idx]
    Xva = X_valid[:, cols_idx]
    Xtr_s = scaler.fit_transform(Xtr)
    Xva_s = scaler.transform(Xva)
    records = []
    best_auc = -np.inf

    for mdl_name, (base, grid) in models_dict.items():
        keys, values = list(grid.keys()), list(grid.values())
        if len(keys)==0:
            hp_list = [dict()]
        else:
            hp_list = [dict(zip(keys, v)) for v in product(*values)]
        for hp in hp_list:
            for seed in seeds:
                # Clonar
                import copy
                clf = copy.deepcopy(base)
                for k,v in hp.items():
                    setattr(clf, k, v)
                # Heurística: usar escala para SVM/MLP/KNN/LogReg
                use_scaled = mdl_name in ['SVM','MLP','KNN','LogReg']
                auc = fit_and_score(clf, Xtr_s if use_scaled else Xtr, y_train, Xva_s if use_scaled else Xva, y_valid, use_scaled)
                best_auc = max(best_auc, auc)
                records.append({'Subset': S_name, 'Model': mdl_name, 'Params': hp, 'Seed': seed,
                                'AUC': auc, 'Complexity': np.nan})
                # Guardar complejidad luego de refit completo
                try:
                    clf.fit(Xtr_s if use_scaled else Xtr, y_train)
                    comp = model_complexity(clf)
                    records[-1]['Complexity'] = comp
                except Exception:
                    pass
    res = pd.DataFrame(records)
    res['BestAUC'] = best_auc
    res['WithinEps'] = res['AUC'] >= (best_auc - epsilon)
    return res

epsilon = 0.01  # tolerancia en AUC absoluta
results_all = []
for key, feats in S_map.items():
    res = build_rashomon_set(key, feats, epsilon=epsilon)
    results_all.append(res)
results_all = pd.concat(results_all, ignore_index=True)
results_all.sort_values(['Subset','AUC'], ascending=[True,False]).head()

## 6) Selección del ε‑Rashomon set y métricas de volumen + complejidad


In [ ]:
rashomon_sets = {s: results_all[(results_all['Subset']==s) & (results_all['WithinEps'])].copy() for s in S_map.keys()}
summary = []
for s, df_s in rashomon_sets.items():
    cnt = len(df_s)
    comp_mean = df_s['Complexity'].dropna().mean() if len(df_s['Complexity'].dropna())>0 else np.nan
    comp_min = df_s['Complexity'].dropna().min() if len(df_s['Complexity'].dropna())>0 else np.nan
    auc_best = df_s['AUC'].max() if len(df_s)>0 else np.nan
    summary.append({'Subset': s, 'RashomonVolume(count)': cnt, 'ComplexityMean': comp_mean, 'ComplexityMin': comp_min, 'BestAUC_in_set': auc_best})
summary_df = pd.DataFrame(summary).sort_values('Subset')
summary_df

## 7) SHAP & LIME sobre el Rashomon set (estabilidad explicativa)
Se calcula **SHAP global** (|SHAP| medio) por modelo y **LIME local** en un conjunto de instancias de validación.

> Si SHAP/LIME no están instalados, se omiten estas secciones.


In [ ]:
def get_shap_global_vector(model, Xbg, Xeval, feature_list):
    if not SHAP_AVAILABLE:
        return None
    try:
        label = model.__class__.__name__
        if label in ['RandomForestClassifier','ExtraTreesClassifier','GradientBoostingClassifier','DecisionTreeClassifier','AdaBoostClassifier','BaggingClassifier']:
            explainer = shap.TreeExplainer(model)
            sv = explainer.shap_values(Xeval)
            if isinstance(sv, list):
                sv = sv[1] if len(sv)>1 else sv[0]
        elif label in ['LogisticRegression']:
            explainer = shap.LinearExplainer(model, Xbg)
            sv = explainer.shap_values(Xeval)
        elif label in ['SVC','KNeighborsClassifier','QuadraticDiscriminantAnalysis','MLPClassifier'] or label.startswith('XGB'):
            explainer = shap.KernelExplainer(model.predict_proba, Xbg)
            sv = explainer.shap_values(Xeval, nsamples=200)
            if isinstance(sv, list):
                sv = sv[1] if len(sv)>1 else sv[0]
        else:
            return None
        g = np.abs(sv).mean(axis=0)
        return pd.Series(g, index=feature_list)
    except Exception as e:
        return None

def kendall_tau_rank_sims(vectors_df):
    # Kendall τ promedio entre rankings (global SHAP)
    if vectors_df is None or len(vectors_df)==0:
        return np.nan
    from scipy.stats import kendalltau
    cols = vectors_df.columns.tolist()
    taus = []
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            r1 = vectors_df[cols[i]].rank(ascending=False)
            r2 = vectors_df[cols[j]].rank(ascending=False)
            tau, _ = kendalltau(r1, r2)
            if not np.isnan(tau):
                taus.append(tau)
    return float(np.mean(taus)) if len(taus)>0 else np.nan

def jaccard_topk(vectors_df, k=5):
    if vectors_df is None or len(vectors_df)==0:
        return np.nan
    cols = vectors_df.columns.tolist()
    from itertools import combinations
    js = []
    for a,b in combinations(cols, 2):
        A = set(vectors_df[a].sort_values(ascending=False).index[:k])
        B = set(vectors_df[b].sort_values(ascending=False).index[:k])
        inter = len(A & B)
        union = len(A | B)
        js.append(inter/union if union>0 else np.nan)
    return float(np.nanmean(js)) if len(js)>0 else np.nan

def ed_l2_mean(vectors_df):
    # Distancia L2 promedio entre vectores globales SHAP normalizados
    if vectors_df is None or len(vectors_df)==0:
        return np.nan
    mats = vectors_df.to_numpy().T
    mats = mats / (np.linalg.norm(mats, axis=1, keepdims=True)+1e-12)
    ds = []
    for i in range(mats.shape[0]):
        for j in range(i+1, mats.shape[0]):
            ds.append(np.linalg.norm(mats[i]-mats[j]))
    return float(np.mean(ds)) if len(ds)>0 else np.nan

def compute_explainability_stability_for_subset(S_name, features, max_models=8, k_top=5):
    if not SHAP_AVAILABLE:
        return {'Subset': S_name, 'KendallTau': np.nan, 'Jaccard@k': np.nan, 'ED_L2': np.nan}
    cols_idx = [feature_names.index(f) for f in features]
    Xtr = X_train[:, cols_idx]
    Xva = X_valid[:, cols_idx]
    # background para KernelExplainer
    bg_rows = min(200, Xtr.shape[0])
    Xbg = shap.sample(Xtr, bg_rows, random_state=RANDOM_STATE) if SHAP_AVAILABLE else Xtr[:bg_rows]
    # tomar hasta max_models del rashomon set
    df_set = rashomon_sets[S_name].copy()
    if len(df_set)==0:
        return {'Subset': S_name, 'KendallTau': np.nan, 'Jaccard@k': np.nan, 'ED_L2': np.nan}
    df_set = df_set.sort_values('AUC', ascending=False).head(max_models)
    vectors = {}
    for idx, row in df_set.iterrows():
        mdl = row['Model']
        import copy
        base, grid = models_dict[mdl]
        clf = copy.deepcopy(base)
        for k,v in row['Params'].items():
            setattr(clf, k, v)
        use_scaled = mdl in ['SVM','MLP','KNN','LogReg']
        clf.fit((scaler.fit_transform(Xtr) if use_scaled else Xtr), y_train)
        g = get_shap_global_vector(clf, Xbg, Xva, features)
        if g is not None:
            vectors[f"{mdl}_{idx}"] = g
    if len(vectors)==0:
        return {'Subset': S_name, 'KendallTau': np.nan, 'Jaccard@k': np.nan, 'ED_L2': np.nan}
    V = pd.DataFrame(vectors)
    return {
        'Subset': S_name,
        'KendallTau': kendall_tau_rank_sims(V),
        'Jaccard@k': jaccard_topk(V, k=k_top),
        'ED_L2': ed_l2_mean(V)
    }

stab_metrics = [compute_explainability_stability_for_subset(s, feats, max_models=8, k_top=5) for s,feats in S_map.items()]
stab_df = pd.DataFrame(stab_metrics)
stab_df

## 8) LIME (opcional) — varianza local
Estimación de **varianza LIME** para un puñado de instancias; útil como control de fragilidad local.

In [ ]:
def lime_local_variance(S_name, features, n_instances=20, repeats=5):
    if not LIME_AVAILABLE:
        return {'Subset': S_name, 'LIME_VarMean': np.nan}
    cols_idx = [feature_names.index(f) for f in features]
    Xtr = X_train[:, cols_idx]
    Xva = X_valid[:, cols_idx]
    df_set = rashomon_sets[S_name].sort_values('AUC', ascending=False)
    if len(df_set)==0:
        return {'Subset': S_name, 'LIME_VarMean': np.nan}
    row = df_set.iloc[0]
    mdl = row['Model']
    base, grid = models_dict[mdl]
    import copy
    clf = copy.deepcopy(base)
    for k,v in row['Params'].items():
        setattr(clf, k, v)
    use_scaled = mdl in ['SVM','MLP','KNN','LogReg']
    Xtr_u = (scaler.fit_transform(Xtr) if use_scaled else Xtr)
    Xva_u = (scaler.transform(Xva) if use_scaled else Xva)
    clf.fit(Xtr_u, y_train)
    expl = LimeTabularExplainer(Xtr_u, feature_names=features, class_names=['neg','pos'], discretize_continuous=True)
    idxs = np.random.choice(np.arange(Xva_u.shape[0]), size=min(n_instances, Xva_u.shape[0]), replace=False)
    vars_inst = []
    for idx in idxs:
        weights_mat = []
        for r in range(repeats):
            exp = expl.explain_instance(Xva_u[idx], clf.predict_proba, num_features=min(10, len(features)))
            # Convertimos a vector ordenado por features
            w = {k: v for k,v in exp.as_list()}
            vec = np.array([w.get(f, 0.0) for f in features])
            weights_mat.append(vec)
        weights_mat = np.vstack(weights_mat)
        vars_inst.append(weights_mat.var(axis=0).mean())
    return {'Subset': S_name, 'LIME_VarMean': float(np.mean(vars_inst)) if len(vars_inst)>0 else np.nan}

lime_df = pd.DataFrame([lime_local_variance(s, feats) for s,feats in S_map.items()])
lime_df

## 9) Gráficas (matplotlib)
Reglas: **un gráfico por figura** y sin estilos de color explícitos.

In [ ]:
# 9.1 Rashomon curves: AUC vs Complejidad
for s in S_map.keys():
    df_s = rashomon_sets[s]
    if len(df_s)==0:
        continue
    plt.figure(figsize=(6,4))
    plt.scatter(df_s['Complexity'], df_s['AUC'])
    plt.xlabel('Complejidad (heurística)')
    plt.ylabel('AUC (validación)')
    plt.title(f'Rashomon curve — Subconjunto {s}')
    plt.show()

# 9.2 Estabilidad vs Complejidad (ejemplo con KendallTau)
if SHAP_AVAILABLE:
    for s in S_map.keys():
        df_s = rashomon_sets[s]
        if len(df_s)==0:
            continue
        # aquí graficamos AUC vs Complejidad; podrías colorear por rank stability si guardas por-modelo
        plt.figure(figsize=(6,4))
        plt.scatter(df_s['Complexity'], df_s['AUC'])
        plt.xlabel('Complejidad (heurística)')
        plt.ylabel('AUC (validación)')
        plt.title(f'Estabilidad (proxy) vs Complejidad — {s}')
        plt.show()


## 10) Exportar resultados


In [ ]:
outdir = 'rashomon_outputs'
os.makedirs(outdir, exist_ok=True)
results_all.to_csv(os.path.join(outdir, 'models_grid_results.csv'), index=False)
summary_df.to_csv(os.path.join(outdir, 'rashomon_summary.csv'), index=False)
stab_df.to_csv(os.path.join(outdir, 'shap_stability.csv'), index=False)
lime_df.to_csv(os.path.join(outdir, 'lime_variance.csv'), index=False)
print('Archivos exportados en', outdir)

### Notas finales
- Ajusta `epsilon` para controlar el tamaño del \(ε\)-Rashomon set.
- Si cambias la métrica objetivo (p. ej., `log_loss`), ajusta la selección WithinEpsilon.
- Puedes enriquecer la complejidad (conteo de nodos, profundidad exacta, #coeficientes no nulos) y añadir **Rashomon ratio** si defines un denominador común del espacio de hipótesis.
- Para datasets con categorías, agrega preprocesamiento (One-Hot/Ordinal) antes del ranking.
